In [ ]:
# _*_ select module : windowns & linux_*_ #
# 举一个EchoServer的例子，客户端发送任何内容，服务端会原模原样返回。
import socket
import select
from Queue import Queue
 
#AF_INET指定使用IPv4协议，如果要用更先进的IPv6，就指定为AF_INET6。
#SOCK_STREAM指定使用面向流的TCP协议，如果要使用面向数据包的UCP协议，就指定SOCK_DGRAM。
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.setblocking(False)
#设置监听的ip和port
server_address = ('localhost', 1234)
server.bind(server_address)
#设置backlog为5，client向server发起connect，server accept后建立长连接，
#backlog指定排队等待server accept的连接数量，超过这个数量，server将拒绝连接。
server.listen(5)
#注册在socket上的读事件
inputs = [server]
#注册在socket上的写事件
outputs = []
#注册在socket上的异常事件
exceptions = []
#每个socket有一个发送消息的队列
msg_queues = {}
print "server is listening on %s:%s." % server_address
while inputs:
     #第四个参数是timeout，可选，表示n秒内没有任何事件通知，就执行下面代码
     readable, writable, exceptional = select.select(inputs, outputs, exceptions)
     for sock in readable:
         #client向server发起connect也是读事件，server accept后产生socket加入读队列中
         if sock is server:
             conn, addr = sock.accept()
             conn.setblocking(False)
             inputs.append(conn)
             msg_queues[conn] = Queue()
             print "server accepts a conn."
         else:
             #读取client发过来的数据，最多读取1k byte。
             data = sock.recv(1024)
             #将收到的数据返回给client
             if data:
                 msg_queues[sock].put(data)
                 if sock not in outputs:
                     #下次select的时候会触发写事件通知，写和读事件不太一样，前者是可写就会触发事件，并不一定要真的去写
                     outputs.append(sock)
             else:
                 #client传过来的消息为空，说明已断开连接
                 print "server closes a conn."
                 if sock in outputs:
                     outputs.remove(sock)
                 inputs.remove(sock)
                 sock.close()
                 del msg_queues[sock]
     for sock in writable:
         if not msg_queues[sock].empty():
             sock.send(msg_queues[sock].get_nowait())
         if msg_queues[sock].empty():
             outputs.remove(sock)
     for sock in exceptional:
         inputs.remove(sock)
         if sock in outputs:
             outputs.remove(sock)
         sock.close()
         del msg_queues[sock]
            
'''
select有3个缺点：
1. 每次调用select，都需要把文件描述符集合从用户态拷贝到内核态，当FD很多时开销会很大。
2. 每次调用select后，都需要在内核遍历传递进来的所有文件描述符，这个开销在fd很多时也很大。
3. 文件描述符数量有限，默认1024。
'''

In [ ]:
# _*_ select module : only linux_*_ #
'''
    与select的主要区别是select需要为读事件、写事件、异常事件分别创建一个描述符的集合，
因此在轮询的时候，需要分别轮询这三个集合。而poll库只需创建一个集合，在每个描述符对
应的结构上分别设置读事件，写事件和异常事件，最后轮询的时候可以同时检查这三种事件是
否发生。
    是select库优化的实现：解决了FD限制，但是失去了跨平台特性
'''
import select
import socket
import sys
import Queue
 
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.setblocking(False)
server_address = ('localhost', 1234)
server.bind(server_address)
server.listen(5)
print 'server is listening on %s port %s' % server_address
msg_queues = {}
timeout = 1000 * 60
#POLLIN: There is data to read
#POLLPRI: There is urgent data to read
#POLLOUT: Ready for output
#POLLERR: Error condition of some sort
#POLLHUP: Hung up
#POLLNVAL: Invalid request: descriptor not open
READ_ONLY = select.POLLIN | select.POLLPRI | select.POLLHUP | select.POLLERR
READ_WRITE = READ_ONLY | select.POLLOUT
poller = select.poll()
#注册需要监听的事件
poller.register(server, READ_ONLY)
#文件描述符和socket映射
fd_to_socket = { server.fileno(): server}
while True:
     events = poller.poll(timeout)
     for fd, flag in events:
         sock = fd_to_socket[fd]
         if flag & (select.POLLIN | select.POLLPRI):
             if sock is server:
                 conn, client_address = sock.accept()
                 conn.setblocking(False)
                 fd_to_socket[conn.fileno()] = conn
                 poller.register(conn, READ_ONLY)
                 msg_queues[conn] = Queue.Queue()
             else:
                 data = sock.recv(1024)
                 if data:
                     msg_queues[sock].put(data)
                     poller.modify(sock, READ_WRITE)
                 else:
                     poller.unregister(sock)
                     sock.close()
                     del msg_queues[sock]
         elif flag & select.POLLHUP:
             poller.unregister(sock)
             sock.close()
             del msg_queues[sock]
         elif flag & select.POLLOUT:
             if not msg_queues[sock].empty():
                 msg = msg_queues[sock].get_nowait()
                 sock.send(msg)
             else:
                 poller.modify(sock, READ_ONLY)
         elif flag & select.POLLERR:
             poller.unregister(sock)
             sock.close()
             del msg_queues[sock]